<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/802%20code/bert-paradetox-1token-mask-evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets transformers

  Using cached huggingface_hub-0.19.4-py3-none-any.whl (311 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.16.4
    Uninstalling huggingface-hub-0.16.4:
      Successfully uninstalled huggingface-hub-0.16.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unbabel-comet 2.2.0 requires huggingface-hub<0.17.0,>=0.16.0, but you have huggingface-hub 0.19.4 which is incompatible.


In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch

# Load the dataset
dataset = load_dataset("HamdanXI/paradetox-1Token-Split-MASK")

# Load the tokenizer and model
model_name = "HamdanXI/bert-base-uncased-paradetox-1Token-Split-MASK-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

# Function to predict the masked token
def predict_masked_token(sentence):
    inputs = tokenizer(sentence, return_tensors="pt")
    mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]

    token_logits = model(**inputs).logits
    mask_token_logits = token_logits[0, mask_token_index, :]
    top_token = torch.argmax(mask_token_logits).item()

    predicted_token = tokenizer.decode([top_token])
    return predicted_token

predicted_sentences = []

# Iterate over the test split and predict
for row in dataset['test']:
    sentence_with_mask = row['input']
    predicted_token = predict_masked_token(sentence_with_mask)
    predicted_sentence = sentence_with_mask.replace(tokenizer.mask_token, predicted_token)
    predicted_sentences.append(predicted_sentence)

In [ ]:
!pip install bert-score
from bert_score import score

In [17]:
def bert_score_evaluate(predictions, label):
  # Compute BERT Score
  P, R, F1 = score(predictions, label, lang="en", rescale_with_baseline=True)

  # Compute and print the average scores
  print(f"Precision: {P.mean()}, Recall: {R.mean()}, F1 Score: {F1.mean()}")

In [18]:
bert_score_evaluate(predicted_sentences, dataset['test']['labels'])

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Precision: 0.8262403011322021, Recall: 0.860024094581604, F1 Score: 0.8430630564689636


In [ ]:
!pip install git+https://github.com/google-research/bleurt.git
from datasets import load_metric

bleurt_metric = load_metric('bleurt')

def bleurt_evaluate(predictions, label):
    results = bleurt_metric.compute(predictions=predictions, references=label)

    scores = results['scores']

    average_score = sum(scores) / len(scores) if scores else 0
    print(f"Average BLEURT Score: {average_score}")

In [20]:
bleurt_evaluate(predicted_sentences, dataset['test']['labels'])

Average BLEURT Score: 0.535543645969816


In [3]:
!pip install evaluate
!pip install unbabel-comet

from evaluate import load
comet_metric = load('comet')

  Using cached huggingface_hub-0.16.4-py3-none-any.whl (268 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.19.4
    Uninstalling huggingface-hub-0.19.4:
      Successfully uninstalled huggingface-hub-0.19.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.15.0 requires huggingface-hub>=0.18.0, but you have huggingface-hub 0.16.4 which is incompatible.


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.1.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/371e9839ca4e213dde891b066cf3080f75ec7e72/checkpoints/model.ckpt`
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/saving.py:177: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [4]:
def comet_evaluate(predictions, input, label):
    comet_score = comet_metric.compute(predictions=predictions, references=label, sources=input)
    print(comet_score)

In [5]:
comet_evaluate(predicted_sentences, dataset['test']['input'], dataset['test']['labels'])

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


{'mean_score': 0.8789231684587152, 'scores': [0.8539836406707764, 0.9236643314361572, 0.7047179937362671, 0.8876857161521912, 0.8162493109703064, 0.9220520257949829, 0.8885858654975891, 0.9743451476097107, 0.799439549446106, 0.8866257071495056, 0.9540861248970032, 0.8732778429985046, 0.6877567172050476, 0.8983725309371948, 0.9311358332633972, 0.9309287667274475, 0.9267333149909973, 0.7653626799583435, 0.928642213344574, 0.8210980296134949, 0.8183625340461731, 0.9103133082389832, 0.7928244471549988, 0.9236517548561096, 0.8793492913246155, 0.947155773639679, 0.8539526462554932, 0.9126984477043152, 0.877565324306488, 0.9209070801734924, 0.9269729256629944, 0.8111934065818787, 0.7757245302200317, 0.9020347595214844, 0.905279815196991, 0.9771950840950012, 0.9664483666419983, 0.882582426071167, 0.8829454779624939, 0.8463860750198364, 0.8683552145957947, 0.901214063167572, 0.9119177460670471, 0.8243674039840698, 0.9326827526092529, 0.9300044178962708, 0.9304800629615784, 0.5399470925331116, 0